# Eval IOU on yolo

In [1]:
import matplotlib.pylab as plt
%matplotlib inline
import cv2
from glob import glob

import pandas as pd
import numpy as np
from tqdm import tqdm

In [2]:
# metric
def get_iou(boxA, boxB):
    """
    Calculate the Intersection over Union (IoU) of two bounding boxes.
    Parameters
    ----------
    boxA = np.array( [ xmin,ymin,xmax,ymax ] )
    boxB = np.array( [ xmin,ymin,xmax,ymax ] )
    Returns
    -------
    float
    in [0, 1]
    """

    bb1 = dict()
    bb1['x1'] = boxA[0]
    bb1['y1'] = boxA[1]
    bb1['x2'] = boxA[2]
    bb1['y2'] = boxA[3]

    bb2 = dict()
    bb2['x1'] = boxB[0]
    bb2['y1'] = boxB[1]
    bb2['x2'] = boxB[2]
    bb2['y2'] = boxB[3]

    # Determine the coordinates of the intersection rectangle
    x_left = max(bb1['x1'], bb2['x1'])
    y_top = max(bb1['y1'], bb2['y1'])
    x_right = min(bb1['x2'], bb2['x2'])
    y_bottom = min(bb1['y2'], bb2['y2'])

    if x_right < x_left or y_bottom < y_top:
        #print('check')
        return 0.0

    # The intersection of two axis-aligned bounding boxes is always an
    # axis-aligned bounding box
    intersection_area = (x_right - x_left) * (y_bottom - y_top)

    # Compute the area of both bounding boxes area
    bb1_area = (bb1['x2'] - bb1['x1']) * (bb1['y2'] - bb1['y1'])
    bb2_area = (bb2['x2'] - bb2['x1']) * (bb2['y2'] - bb2['y1'])

    # Compute the intersection over union by taking the intersection
    # area and dividing it by the sum of prediction + ground-truth
    # areas - the intersection area
    iou = intersection_area / float(bb1_area + bb2_area - intersection_area)

    assert iou >= 0.0
    assert iou <= 1.0

    return iou

In [3]:
face_cascade = cv2.CascadeClassifier('../v0_prototype/data/haarcascade_frontalface_default.xml')
path = '../../../WIDER_val/images/*/*.jpg'
labels = pd.read_csv('../../../wider_face_split/wider_face_val_bbx_gt.txt', header=None)

In [5]:
sum_iou = 0
cnt = 0
for filename in tqdm(glob(path)):
    # read data
    img = cv2.imread(filename)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.1, 4)
    name = filename[len('../../../WIDER_val/images/'):]

    # predict
    for (x, y, w, h) in faces:
        cv2.rectangle(img, (x, y), (x+w, y+h), (0, 0, 255), 2)
    y_pred = [[int(i) for i in list(j)] for j in faces]

    # fact
    ind_lab = labels[labels[0] == name].index[0]
    num_faces = int(labels.iloc[ind_lab+1][0])
    y_true = np.array(list(labels.iloc[ind_lab+2:ind_lab+num_faces+2][0] \
        .apply(lambda x: x.split()[:4]).apply(lambda x: [int(i) for i in x])))
    
    # correct for metric
    for i in range(len(y_pred)):
        y_pred[i][2] = y_pred[i][2] + y_pred[i][0]
        y_pred[i][3] = y_pred[i][3] + y_pred[i][1]
    for i in range(len(y_true)):
        y_true[i][2] = y_true[i][2] + y_true[i][0]
        y_true[i][3] = y_true[i][3] + y_true[i][1]
        
    # evaluate
    total_iou = 0
    pred_dict = dict()
    for gt in y_true:
        max_iou_per_gt = 0
        for i, pred in enumerate(y_pred):
            if i not in pred_dict.keys():
                pred_dict[i] = 0
            iou = get_iou(gt, pred)
            if iou > max_iou_per_gt:
                max_iou_per_gt = iou
            if iou > pred_dict[i]:
                pred_dict[i] = iou
        total_iou = total_iou + max_iou_per_gt
    iou = total_iou / len(y_true)
    
    sum_iou += iou
    cnt += 1
total_iou = sum_iou / cnt
total_iou

100%|███████████████████████████████████████| 3226/3226 [19:27<00:00,  2.76it/s]


0.2837784039271598